In [ ]:
# Используется временно для отладки
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
import cv2
import glob
import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import torch
import torchvision
from torchvision.ops.boxes import nms

In [ ]:
# Вариант 1
def predict_avg(path_to_files):
    """
    Для предсказания среднего числа бластоспор в папке path_to_files нужно создать две папки:
    images - для фотографий в формате jpg и model - для модели библиотеки pytorch

    """
    def predict_boxes(path_to_image, 
                      model,
                      iou_threshold=0.1, 
                      threshold=0.5):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.eval()
        img = cv2.imread(path_to_image)
        img_ = img / 255.
        img_ = torch.from_numpy(img_).permute(2, 0, 1).unsqueeze(0).to(torch.float).to(device)
        predict = model(img_)
        ind = nms(predict[0]['boxes'], predict[0]['scores'], iou_threshold).detach().cpu().numpy()
    
        boxes_aft_threshold = []

        for i, box in enumerate(predict[0]['boxes'][ind]):
            if predict[0]['scores'][i] > threshold:
                boxes_aft_threshold.append(predict[0]['boxes'][i].cpu().detach().numpy())
        
        return boxes_aft_threshold


    def draw_boxes(path_to_image, 
                   boxes):
        plt.rcParams["figure.figsize"] = (12,8)
        with Image.open(path_to_image) as image:
            plt.imshow(image)

        for item in boxes:
            width_img = int(item[2] - item[0])
            height_img = int(item[3] - item[1])
            ax = plt.gca()
            rect = patches.Rectangle((item[0],item[1]), 
                                     width_img, 
                                     height_img, 
                                     linewidth=1, 
                                     edgecolor='red', 
                                     fill = False)
            ax.add_patch(rect)
        plt.figure(figsize=(12,10))
        plt.show()


    def calc_numb_blastospores(path_to_image, model, draw=True):
        boxes = predict_boxes(path_to_image, model)
        numb_blastospores = len(boxes)
        print(f"filename: {path_to_image.split('/')[-1]}", ' | ' 
              f'Предсказанное число бластоспор: {numb_blastospores}')
        if draw:
            draw_boxes(path_to_image, boxes)
        
        return numb_blastospores 


    def data_prediction(path_to_images, model):
        avg_numb = []
        images_paths = glob.glob(os.path.join(path_to_images, '*.jpg'))
        for imh_path in images_paths:
            avg_numb.append(calc_numb_blastospores(imh_path, model, draw=False))
        return np.mean(avg_numb)
    
    model = torch.load(os.path.join(path_to_files, 'model', 'full_frcnn_resnet50_fpn.pth'))
    result = data_prediction(path_to_images=os.path.join(path_to_files, 'images'),
                             model=model)

    return f'Среднее число бластоспор: {result}'

predict_avg('/content/drive/MyDrive/Misha/All_data_YOLO/test_predict')

filename: 451.jpg  | Предсказанное число бластоспор: 100
filename: 452.jpg  | Предсказанное число бластоспор: 100
filename: 453.jpg  | Предсказанное число бластоспор: 100
filename: 343.jpg  | Предсказанное число бластоспор: 78
filename: 345.jpg  | Предсказанное число бластоспор: 90
filename: 344.jpg  | Предсказанное число бластоспор: 89


'Среднее число бластоспор: 92.83333333333333'

In [ ]:
# Вариант 2
def predict(path_to_image, model):
    """
    Для предсказания среднего числа бластоспор в папке path_to_files нужно создать две папки:
    images - для фотографий в формате jpg и model - для модели библиотеки pytorch

    """
    def predict_boxes(path_to_image, 
                      model,
                      iou_threshold=0.1, 
                      threshold=0.5):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.eval()
        img = cv2.imread(path_to_image)
        img_ = img / 255.
        img_ = torch.from_numpy(img_).permute(2, 0, 1).unsqueeze(0).to(torch.float).to(device)
        predict = model(img_)
        ind = nms(predict[0]['boxes'], predict[0]['scores'], iou_threshold).detach().cpu().numpy()
    
        boxes_aft_threshold = []
    
        for i, box in enumerate(predict[0]['boxes'][ind]):
            if predict[0]['scores'][i] > threshold:
                boxes_aft_threshold.append(predict[0]['boxes'][i].cpu().detach().numpy())
    
        return boxes_aft_threshold
    
    def draw_boxes(path_to_image, 
                   boxes):
        plt.rcParams["figure.figsize"] = (12,8)
        with Image.open(path_to_image) as image:
            plt.imshow(image)

        for item in boxes:
            width_img = int(item[2] - item[0])
            height_img = int(item[3] - item[1])
            ax = plt.gca()
            rect = patches.Rectangle((item[0],item[1]), 
                                     width_img, 
                                     height_img, 
                                     linewidth=1, 
                                     edgecolor='red', 
                                     fill = False)

            ax.add_patch(rect)

        plt.figure(figsize=(12,10))
        plt.show()

    def calc_numb_blastospores(path_to_image, model, draw=True):
        boxes = predict_boxes(path_to_image, model)
        numb_blastospores = len(boxes)
        print(f"filename: {path_to_image.split('/')[-1]}", ' | ' 
              f'Предсказанное число бластоспор: {numb_blastospores}')
        if draw:
            draw_boxes(path_to_image, boxes) 
        return numb_blastospores 
    
    result = calc_numb_blastospores(path_to_image, model=model, draw=False)
    return result

In [ ]:
def calc_avg(path_to_files):
    avg_numb = []
    model = torch.load(os.path.join(path_to_files, 'model', 'full_frcnn_resnet50_fpn.pth'))
    path_to_images = os.path.join(path_to_files, 'images')
    images_paths = glob.glob(os.path.join(path_to_images, '*.jpg'))
    for imh_path in images_paths:
        avg_numb.append(predict(imh_path, model))
    return f'Среднее число бластоспор: {np.mean(avg_numb)}'

calc_avg('/content/drive/MyDrive/Misha/All_data_YOLO/test_predict')

filename: 451.jpg  | Предсказанное число бластоспор: 100
filename: 452.jpg  | Предсказанное число бластоспор: 100
filename: 453.jpg  | Предсказанное число бластоспор: 100
filename: 343.jpg  | Предсказанное число бластоспор: 78
filename: 345.jpg  | Предсказанное число бластоспор: 90
filename: 344.jpg  | Предсказанное число бластоспор: 89


'Среднее число бластоспор: 92.83333333333333'